In [49]:
import numpy as np
from numpy.linalg import inv

# Ejemplo
## cjt. datos 23, problema PL 1

In [50]:
#matriz de restricciones
A = np.array(
    [[15, -36, 30, -54, 39, -92, 54, -76, 75, 30, 54, -36, -18, 79, 0, 0, 0, 0, 0, 0], 
     [-41, -38, 90, 74, 56, 37, 32, 92, 32, 35, 57, 64, 98, -51, 0, 0, 0, 0, 0, 0], 
     [-48, 38, -18, 20, 89, 15, 56, -64, -78, 35, 65, 55, -87, -23, 0, 0, 0, 0, 0, 0],
     [-65, 36, 77, 49, 74, 68, 56, -91, 37, 78, 9, 63, -36, -63, 0, 0, 0, 0, 0, 0],
     [100, 93, 50, 86, 50, 81, 67, 52, 52, 87, 61, 72, 68, 97, 1, 0, 0, 0, 0, 0],
     [-23, -27, -69, 40, -93, 48, -11, -51, 64, -82, 0, 47, 65, 97, 0, 1, 0, 0, 0, 0],
     [90, 18, 51, -78, 2, 98, -11, 74, -1, -7, 45, 70, 88, 1, 0, 0, 1, 0, 0, 0],
     [-3, 95, -57, -18, -33, 100, -18, 85, 84, -17, 42, 29, -52, 74, 0, 0, 0, 1, 0, 0],
     [47, 24, -19, 89, -74, 77, 13, 60, 83, 66, -17, -24, -3, 58, 0, 0, 0, 0, 1, 0],
     [-75, 43, 46, 72, 21, -99, 97, 92, 82, -25, 41, 50, -5, 57, 0, 0, 0, 0, 0, 1]
     ])

#coeficientes de la función objetivo
c = np.array([44,18,60,27,-79,-51,92,-78,-69,-35,26,-56,-10,-55,0,0,0,0,0,0])

#términos independientes de las restricciones
b = np.array([64, 537, 55, 292, 1017, 6, 441, 312, 381, 398])

# Fase I (problema artificial)
## Usamos Fase II para resolver el problema

**1. Inicialización**

· La función objetivo auxiliar de la fase I consiste en minimizar la suma de las variables artificiales. Por lo tanto el **vector de costos (c)** de la fase I consistirá en coeficiente 1 en las variables artificiales y coeficientes 0 en las originales y las de holgura.

· La **matriz de restricciones (A)** de la fase I será la matriz que nos dan, seguida de una matriz identidad asociada a las variables artificiales


In [51]:
# formato fase 1

m=len(A) # num de restricciones
n=len(A[0]) # num de variables
# num de variables basicas = n-m // num de variables no basicas = m

id=np.identity(m) #matriz identidad (la usamos para crear la A de la fase I)
ones=np.ones(m) #vector de 1 (lo usamos para crear la c de la fase I)

indices_basicas = np.array(range(n,n+m)) 
indices_no_basicas = np.array(range(n)) 
indices_inventadas = np.array(range(n,n+m)) #al inicio de la fase I, las basicas son las variables inventadas

# c fase 1
c=np.zeros(n) 
c = np.insert(c,n,ones) #c = vector de 0 (variables originales) + vector de 1 (variables artificiales)
cn = c[indices_no_basicas]
cb = c[indices_basicas]

# A fase 1 
A = np.insert(A,n,id,axis=1) #A = matriz de restricciones original + matriz identidad (variables artificiales)
filas=np.array(range(m))
B = A[np.ix_(filas,indices_basicas)]
An = A[np.ix_(filas,indices_no_basicas)]
#B_inv = inv(B) -> 1a iteració de la fase I B_inv = B
B_inv = B
# Cálculo de Xb
# Xb=B_inv*b -> 1a iteración de la fase I Xb = b
Xb = np.copy(b)
#si en Xb todos los valores son mayores o iguales a 0, es una SBF
if all(val>=0 for val in Xb):
    print('Es SBF')

z = cb @ Xb #cb*Xb

Es SBF


# 1a ITERACIÓN

**2. Identificación de SBF óptima y selección de variable de entrada:**
1. Calcular los costes reducidos 𝑟
2. Si 𝑟 ≥ 0 entonces la actual SBF es óptima → STOP. Sino, seleccionar una variable no básica 𝑞 con 𝑟𝑞′ < 0 (variable de entrada)

In [52]:
# r = Cn - Cb * inv(B) * An
cb_x_invB = cb @ B_inv  # Multiplicación de cb por la inversa de B
cb_x_invB_An = cb_x_invB @ An  # Multiplicación del resultado anterior por An
r = cn - cb_x_invB_An 

print(f"r = {r}")

# Selección de la variable de entrada usando la regla de Bland
# elegimos la variable no básica de entrada con el subindice más pequeño con coste negativo 
optimo = True
q_idx = None
for i in range(len(r)):
    # Como ordenamos las variables no básicas por los subíndices, elegimos el primer valor negativo. Si no hay valores negativos, quiere decir que es optimo
    if r[i]<0:
        q_idx=i #subindice de la variable de entrada
        optimo = False
        break

if optimo:
    print("SBF optimo fase I encontrado")
    # Aquí en la fase I comprovaremos que ningun indices pertenece a una variable artificial y que la z sea cero
    if not np.isclose(0,z):
        print(f"No factible")
    else:
        if np.any(np.isin(indices_basicas, indices_inventadas)):
            print("Variable artificial aparece en la SBF inicial para la Fase II")
        else:
            print("Ninguna variable artificial aparece en la SBF inicial para la Fase II")
else:
    print("No es el óptimo, seguimos buscando")
q = indices_no_basicas[q_idx]
print(f"q = {q}" if q_idx is not None else "")            

r = [   3. -246. -181. -280. -131. -333. -335. -173. -430. -200. -357. -390.
 -118. -326.   -1.   -1.   -1.   -1.   -1.   -1.]
No es el óptimo, seguimos buscando
q = 1


**3. Cálculo de un DBF de descenso:**
1. Calcular 𝑑𝐵 = −inv(𝐵)*𝐴𝑞 (DBF asociada a 𝑥𝑞)
2. Si 𝑑𝐵 ≥ 0 ⟹ DBF de descenso no acotada⟹ 𝑃𝐿 no acotado → STOP

In [53]:
Aq = A[:, q]  # Extraemos la columna correspondiente a xq en A
db = -B_inv @ Aq  # Calculamos la dB

# Si todos los valores en db son mayores o iguales a 0, el problema no está acotado
if np.all(db >= 0):  
    print('(PL) no acotado')  
else:
    print("(PL) acotado")

print(f"Aq = {Aq}")
print(f"db = {db}")


(PL) acotado
Aq = [-36 -38  38  36  93 -27  18  95  24  43]
db = [ 36  38 -38 -36 -93  27 -18 -95 -24 -43]


**4. Cálculo de la longitud de paso máximo y selección de la variable de sálida:**
1. Escoger la 𝜃* 
2. Variable básica de salida: 𝐵 𝑝 tal que 𝜃* = −𝑥ℬ 𝑝 /𝑑ℬ(𝑝)

In [54]:
# Calculamos las 𝜃 (donde db(i) és negativo) y escogemos la mínima, 
# si db(i) es postivo, ponemos un valor infinito para no escoger esa 𝜃
thetas_lst = np.where(db < -1e-10, -Xb / db, np.inf) 
print(f"𝜃 = {thetas_lst}")
theta = np.min(thetas_lst) # Escogemos el valor mínimo 
p = np.argmin(thetas_lst)
print(f"𝜃* = {theta}")
print(f"p = {p}")


𝜃 = [        inf         inf  1.44736842  8.11111111 10.93548387         inf
 24.5         3.28421053 15.875       9.25581395]
𝜃* = 1.4473684210526316
p = 2


**5. Actualizaciones y cambio de base:**
1. Actualizar las variables básicas y la función objetivo: x𝐵 ≔ xB + 𝜃𝑑𝐵, 𝑥𝑞 ≔ 𝜃 ; 𝑧 = 𝑧 + 𝜃*𝑟𝑞
2. Actualizar los conjuntos ℬ y 𝒩: ℬ:=ℬ∖{𝐵(𝑝)} ∪ {𝑞} , 𝒩 ≔ 𝒩 ∖ {𝑞} ∪ {𝐵(p)}

In [55]:
# Actualizamos índices para las variables básicas
temp = indices_basicas[p]
indices_basicas[p] = q

# Actualizar índices para las variabels no básicas (quitar q y añadir el que salió)
indices_no_basicas = np.array([i for i in range(A.shape[1]) if i not in indices_basicas])

# Actualizar las matrices B y An con los nuevos ínidices
B = A[:, indices_basicas]
An = A[:, indices_no_basicas]

#Actualizar cb y cn
cb = c[indices_basicas]
cn = c[indices_no_basicas]

# Actualizar Xb 
Xb_actual = np.zeros(m)
for i in range(m):
    if i == p:
        Xb_actual[i] = theta
    else:
        Xb_actual[i] = Xb[i] + theta * db[i]
Xb = Xb_actual

# Verificar factibilidad
if np.any(Xb_actual < -1e-10):  
    print("Solución no factible")

# Actualizar z
z_nuevo = float(z) + r[q_idx] * theta
#Comprovar que la z nueva es menor que la z anterior
if (z_nuevo < z):
    print(f"La z nueva mejora -> z_nueva = {z_nuevo}  z_anterior = {z}")
else:
    print(f"La z nueva NO mejora -> z_nueva = {z_nuevo}  z_anterior = {z}")
z = z_nuevo

#Actualización de la B_inv

# Calcular y_k = B_inv * a_k (columna que entra)
a_k = A[:, q]       # Se obtiene la columna 'q' de la matriz A (variable entrante)
y_k = B_inv @ a_k   # Se calcula y_k = B_inv * a_k

# Construir la matriz de transformación E
E = np.eye(m)               # Inicializamos E como una matriz identidad de tamaño m×m
E[:, p] = -y_k / y_k[p]    # Reemplazamos la columna 'p' de E con -y_k / y_k[p]
E[p, p] = 1 / y_k[p]       # Ajustamos el elemento diagonal E[p,p] para pivotear

# Actualizar B_inv usando E
B_inv_actualizada = E @ B_inv

#Comprobamos si se actualiza bien la inversa de B
if np.allclose(np.linalg.inv(B), B_inv_actualizada):
    print("Inversa de B correctamente actualizada")
    B_inv = B_inv_actualizada
else:
    print("Inversa de B no se actualiza correctamente, volvemos a calcular la inversa desde cero")
    B_inv = np.linalg.inv(B)

La z nueva mejora -> z_nueva = 3146.9473684210525  z_anterior = 3503.0
Inversa de B correctamente actualizada


**6. Ir a 2.**


# N-sima iteración:
**Si continuamos ejecutando el simplex, estas son los resultados de las iteraciones futuras:**
| Iteración|    q        |      B(p)      |    θ*   |       z      |
|----------|-------------|----------------|---------|--------------|
| 1        | 1           | 22             | 1.447   | 3503.000000  |
| 2        | 0           | 27             | 1.491   | 3146.947368  |
| 3        | 2           | 23             | 2.923   | 2687.972335  |
| 4        | 3           | 24             | 1.559   | 1726.204558  |
| 5        | 5           | 28             | 0.258   | 1316.888708  |
| 6        | 6           | 26             | 0.751   | 1283.464975  |
| 7        | 7           | 29             | 1.099   | 1202.171457  |
| 8        | 4           | 21             | 1.541   | 798.085558   |
| 9        | 8           | 20             | 1.809   | 764.130826   |
| 10       | 10          | 6              | 2.743   | 306.865470   |
| 11       | 11          | 25             | 1.558   | 176.845578   |
| 12       | -           | -              | -       | -0.000000    |

SBF encontrada en la iteración 12.
